In [1]:
import pandas as pd
import sys
import csv

In [3]:
TAGS_DIR = "../Data/Tags.csv"
QNS_DIR = "../Data/Questions.csv"
output_file = "StackSample"

In [4]:
tags_df = pd.read_csv(TAGS_DIR, sep=',')
print(len(tags_df), tags_df.head())

3750994    Id             Tag
0  80            flex
1  80  actionscript-3
2  80             air
3  90             svn
4  90     tortoisesvn


In [ ]:
top_tag_map = tags_df.groupby('Tag', as_index=False).agg(lambda x: set(x))
top_tag_map

In [ ]:
qn_tag_map = tags_df.groupby('Id', as_index=True).agg(lambda x: set(x))
qn_tag_map

In [ ]:
top_tag_map["Tag_count"] = top_tag_map.apply(lambda row: len(row.Id), axis = 1)

In [ ]:
top_tag_map_100 = top_tag_map.sort_values("Tag_count", ascending=False).head(100).reset_index(drop=True)
top_tag_map_100

In [ ]:
all_id = set()
for i in range(0, 100):
    print(top_tag_map_100.loc[i]['Tag'], len(top_tag_map_100.loc[i]['Id']))
    all_id.update(top_tag_map_100.loc[i]['Id'])
print(len(all_id))

In [ ]:
def get_qns():
    with open(QNS_DIR, "rt", newline='', encoding='ISO-8859-1') as qns_file:
        datareader = csv.DictReader(qns_file)
        try:
            for row in datareader:
                if(int(row["Id"]) in all_id):
                    tags = list(qn_tag_map.at[int(row["Id"]), 'Tag'])
                    yield [row["Title"], row["Body"], tags, len(tags)]
        except:
            pass

In [ ]:
data = []
qn_generator = get_qns()

In [ ]:
ids = [250000, 500000, 750000, len(all_id)]

In [ ]:
i = 0
for n, idx in enumerate(ids): 
    data = []
    while(i < idx):
        try:
            data.append(next(qn_generator))
            print(i)
            i += 1
        except Exception as e:
            print(e)
            break
    df = pd.DataFrame(data, columns = ['Title', 'Body', 'Tags', 'Tag Count'])
    df.to_csv(f'{output_file}{n+1}.csv', sep=";", encoding='utf-8-sig', index_label='ID')